In [39]:
import os
import pandas as pd
import re
import string

In [14]:
def read_txt_to_str(path:str)->str:
    data = ''
    try:
        my_file = open(path, "r")
        data = my_file.read().replace('\n', ' ')
        my_file.close()
        return data
    except Exception as e:
        print(f'error {path}, {e}')
        return data

In [31]:
folder_name = ['business','entertainment','politics','sport','tech']
res = {}
temp_to_df = []
for folder in folder_name:
    dir_path = os.path.join(os.getcwd(),'news_data',folder)
    res_temp = []
    for path in os.listdir(dir_path):
        if os.path.isfile(os.path.join(dir_path, path)):
            str_res = read_txt_to_str(os.path.join(dir_path,path))
            res_temp.append(str_res)
            temp_to_df.append([str_res,folder])
    res[folder] = res_temp 

error c:\Users\Andrey\Documents\exploration\document_clustering_spending_tracker\document_classification\news_data\sport\199.txt, 'utf-8' codec can't decode byte 0xa3 in position 257: invalid start byte


In [40]:
df = pd.DataFrame(temp_to_df,columns=['text','cat'])

In [42]:
import nltk

nltk.download('wordnet') # for download to this path
nltk.download('punkt')
nltk.data.path.append('corpora')
nltk.data.path.append('tokenizers')

try:
    from nltk.stem.wordnet import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    def lemmatize(text : str) -> str:
        return lemmatizer.lemmatize(text)
except Exception as e:
    print(f'failed to load WordNetLemmatizer {e}')
    def lemmatize(text : str) -> str:
        return text

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Andrey\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andrey\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [43]:
try:
    from nltk.tokenize import word_tokenize
except Exception as e:
    print(f'error load nltk tokenize {e}')
    def word_tokenize(text:str)->list:
        return text.split()

In [44]:
def text_cleaning(text:str)->str:
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

def list_to_text(l:list)->str:
    return ' '.join(l)

In [45]:
def preprocess_text(text : str) -> str:
    res = []
    text = text_cleaning(text)
    list_text = word_tokenize(text)
    for word in list_text:
        res.append(lemmatize(word))
    return list_to_text(res)

In [46]:
df['text_cleaned'] = df['text'].apply(lambda x:preprocess_text(x))

In [49]:
def get_max_len(text:str)->int:
    return len(text.split())

df['max_len'] = df['text_cleaned'].apply(lambda x:get_max_len(x))

In [57]:
unique_mapping = {}
for row_text in df['text_cleaned']:
    for word in row_text.split():
        if word not in unique_mapping:
            unique_mapping[word] = 1
        else:
            unique_mapping[word]+=1

In [59]:
len(unique_mapping)

28095

In [60]:
df.head()

,text,cat,text_cleaned,max_len
0,Ad sales boost Time Warner profit Quarterly p...,business,ad sale boost time warner profit quarterly pro...,399
1,Dollar gains on Greenspan speech The dollar h...,business,dollar gain on greenspan speech the dollar ha ...,377
2,Yukos unit buyer faces loan claim The owners ...,business,yukos unit buyer face loan claim the owner of ...,260
3,High fuel prices hit BA's profits British Air...,business,high fuel price hit ba profit british airway h...,383
4,Pernod takeover talk lifts Domecq Shares in U...,business,pernod takeover talk lift domecq share in uk d...,252
